In [ ]:
# -*- coding: utf-8 -*-

import os
import json
import fiona
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from collections import defaultdict
from urllib.request import urlopen
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import matplotlib.pyplot as plt
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from impyute.imputation.cs import mice
from sklearn.preprocessing import StandardScaler
from kmodes.kmodes import KModes
import dash_daq as daq
import base64



mapbox_token = 'pk.eyJ1IjoiaXBlcm1hdGFzYXIiLCJhIjoiY2sxdDV4d2txMG5mcTNjazFwcDZnNjd0eCJ9.r-vxN30jyykxIJZ8P8LhGA'

### External data needed

shapefile = 'SA2_Boundaries/1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp'
feature_file = 'sa2_features_filled.csv'
actual_data = 'actual_features.csv'
topic = 'dash_sentiment_topic.json'
sentiment = 'ourT_final_sentiment.json'
cluster_aussie = 'df_kmodes_all.csv'
spatial_file = 'sa2_spatial.csv'
category_all = 'cat_combine.csv'
category_complete = 'cat_combine_all.csv'


data_sa2 =  pd.read_csv(feature_file)
data_sa2 = data_sa2.iloc[:,1:]
full_df = pd.read_csv(actual_data)
full_df = full_df.iloc[:,1:]
df_kmodes_all = pd.read_csv(cluster_aussie)
df_kmodes_all = df_kmodes_all.iloc[:,1:]
sa2_spatial = pd.read_csv(spatial_file)
sa2_spatial = sa2_spatial.iloc[:,1:]
cat_combine = pd.read_csv(category_all)
cat_combine = cat_combine.iloc[:,1:]
cat_combine_all = pd.read_csv(category_complete)
cat_combine_all = cat_combine_all.iloc[:,1:]

map_df = gpd.read_file(shapefile)

keep_sa2 = pd.read_csv('keep_sa2.csv')
keep_sa2 = keep_sa2.iloc[:,1:]
keep_sa2 = keep_sa2.rename(columns={'0':'SA2_NAME'})


data = keep_sa2.set_index('SA2_NAME').join(data_sa2.set_index('SA2_NAME'), how = 'left').reset_index()


### For filtering in Recommendation

state_name = list(set(data.STATE_NAME_2016))
state_option = [{'label':str(index + 1) + " " + item, 'value':item} for index, item in enumerate(state_name)]

sa4_name = {'Australia' : ['Any']}
for state in state_name:
    sa4_name[state] = ['Any'] + [item for item in list(set(data[data.STATE_NAME_2016 == state]['SA4_NAME_2016']))]
    
sa4_option = {}
for state in state_name:
    sa4_option[state] = [{'label':item, 'value':item} for index, item in enumerate(list(set(data[data.STATE_NAME_2016 == state]['SA4_NAME_2016'])))]

sa4_unique = list(set(data.SA4_NAME_2016))     

sa4_name2 = {}
for state in state_name:
    sa4_name2[state] = [item for item in list(set(data[data.STATE_NAME_2016 == state]['SA4_NAME_2016']))]
    
sa2_name = {}
for sa4 in sa4_unique:
    sa2_name[sa4] = [item for item in list(set(data[data.SA4_NAME_2016 == sa4]['SA2_NAME_2016']))]
        

### For city profile
        
full_df.replace('None', np.nan, inplace=True)

for i in range(3,len(full_df.columns)):
    full_df.iloc[:,i] = full_df.iloc[:,i].astype(float)

full_features = list(full_df.columns)
num_features = sorted(list(full_features[3:]))

features_business = [x for x in num_features if x.split("_")[0] == 'business']
features_demography = [x for x in num_features if x.split("_")[0] == 'demography']
features_development = [x for x in num_features if x.split("_")[0] == 'development']
features_education = [x for x in num_features if x.split("_")[0] == 'education']
features_employment = [x for x in num_features if x.split("_")[0] == 'employment']
features_family = [x for x in num_features if x.split("_")[0] == 'family']
features_finance = [x for x in num_features if x.split("_")[0] == 'finance']
features_health = [x for x in num_features if x.split("_")[0] == 'health']
features_housevalue = [x for x in num_features if x.split("_")[0] == 'housevalue']
features_income = [x for x in num_features if x.split("_")[0] == 'income']
features_indigenous = [x for x in num_features if x.split("_")[0] == 'indigenous']
features_language = [x for x in num_features if x.split("_")[0] == 'language']
features_population = [x for x in num_features if x.split("_")[0] == 'population']
features_unitvalue = [x for x in num_features if x.split("_")[0] == 'unitvalue']
features_volunteer = [x for x in num_features if x.split("_")[0] == 'volunteer']


with open(topic,'r') as f:
    df_topic =  json.load(f)
df_topic = pd.DataFrame.from_dict(df_topic, orient='index').reset_index()
df_topic = df_topic[['index','hashtag','topic']]
df_topic = df_topic.rename(columns={'index':'SA2_CODE'})
df_topic['SA2_CODE'] = df_topic['SA2_CODE'].astype('int32')

with open(sentiment,'r') as f:
    df_sentiment =  json.load(f)
df_sentiment = pd.DataFrame.from_dict(df_sentiment, orient='index').reset_index()
df_sentiment = df_sentiment.rename(columns={'index':'SA2_CODE','polarity':'Score'})
df_sentiment['SA2_CODE'] = df_sentiment['SA2_CODE'].astype('int32')


area_level = data[['SA2_CODE','SA3_NAME_2016','SA4_NAME_2016','AREA_ALBERS_SQKM']]

data_all = area_level.set_index('SA2_CODE').join(full_df.set_index('SA2_CODE'), how = 'left').reset_index()
data_twitter = data_all.set_index('SA2_CODE').join(df_sentiment.set_index('SA2_CODE'), how = 'left').reset_index()
data_twitter_all = data_twitter.set_index('SA2_CODE').join(df_topic.set_index('SA2_CODE'), how = 'left').reset_index()



        
        
### For comparison    
    
feature1 = 'population_population_density'
feature2 = 'housevalue_median_sales_price_last_12_months'
feature3 = 'income_income_med_tot_2016'

def get_df(data,feature1,feature2,feature3):
    df_selected = data.copy()
    df_selected = data[['SA2_NAME_2016','STATE_NAME_2016',feature1,feature2,feature3]]
    df_selected = df_selected.rename(columns={'SA2_NAME_2016':'Areas Name','STATE_NAME_2016':'Areas State',feature1: 'Feature1',feature2:'Feature2',feature3:'Feature3'})
    return df_selected

df = get_df(data,feature1,feature2,feature3)

### Make a table of all features
### cat : education, economy, income ...
### features : feature name
### features name : combination cat and feature as original feature name

df_features = pd.DataFrame()
df_features['cat'] = [x.split("_")[0] for x in list(data.iloc[:,14:].columns)]
df_features['features'] =  list(data.iloc[:,14:].columns)     
df_features['features_name'] =  [' '.join(x.split("_")[1:]) for x in list(data.iloc[:,14:].columns)]   

cat_name = list(set(df_features.cat))
features_dict = {}
for col in cat_name:
    features_dict[col] = [item for item in list(set(df_features[df_features.cat == col]['features_name']))]


data['family_married'] = data.apply(lambda x: ((x.family_female_married + x.family_male_married)/2), axis=1) 

data_selected = data.copy()
data_selected['family_married'] = data_selected.apply(lambda x: x.family_married/10, axis=1) 
data_selected['health_life_satisfaction_pc_70_synth'] = data_selected.apply(lambda x: x.health_life_satisfaction_pc_70_synth*100, axis=1)
data_selected['population_population_density'] = data_selected.apply(lambda x: x.population_population_density/10 , axis=1)
data_selected['housevalue_median_sales_price_last_12_months'] = data_selected.apply(lambda x: x.housevalue_median_sales_price_last_12_months/10000 , axis=1)
data_selected['income_income_med_tot_2016'] = data_selected.apply(lambda x: x.income_income_med_tot_2016/1000 , axis=1)
data_selected['employment_employment_total'] = data_selected.apply(lambda x: x.employment_employment_total/1000 , axis=1)

sa2_code_name = data[['SA2_CODE','SA2_NAME_2016']]

feature_name = ['housevalue_median_sales_price_last_12_months','income_income_med_tot_2016',
                 'population_population_density', 'education_persons_with_post_school_qualifications_bachelorDegree_2016',
                 'health_life_satisfaction_pc_70_synth','employment_employment_total',
                 'development_development_on_track_perc_2018','family_married']

column_name = ['SA2_NAME_2016'] + feature_name

feature_rename = [x.split('_')[0].title() for x in feature_name]

column_rename = ['Areas Name'] +  feature_rename

complete_features = list(data.iloc[:,14:].columns)





############################################# DASH LAYOUT  #############################################



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#111111',
    'font_family': 'Helvetica',
    'text': '#0C090A'
}

markdown_text_Tab1_1 = '''
#### Preference Area

'''

markdown_text_Tab1_2 = '''
#### Preference Features

'''

markdown_text_Tab1_3 = '''
#### Recommended Areas

'''

markdown_text_Tab1_4 = '''
#### Map Visualisation Based on Score

'''

markdown_text_Tab1_5 = '''
#### Feature Deviation From Expected Value

'''

markdown_text_Tab1_6 = '''
#### Feature Statistics

'''

markdown_text_Tab1_7 = '''
#### Feature Categories

'''

markdown_text_Tab1_8 = '''
#### Cluster Analysis

'''

markdown_text_Tab1_9 = '''
#### Distribution Over Categories

'''

markdown_text_Tab1_10 = '''
#### Cluster Analysis Australia Level

'''

markdown_text_Tab2_1 = '''
#### Features Comparison

'''

markdown_text_Tab3_1 = '''
#### Housevalue

'''

markdown_text_Tab3_2 = '''
#### Unitvalue

'''

markdown_text_Tab3_3 = '''
#### Demography

'''

markdown_text_Tab3_4 = '''
#### Population

'''

markdown_text_Tab3_5 = '''
#### Income

'''

markdown_text_Tab3_6 = '''
#### Education

'''

markdown_text_Tab3_7 = '''
#### Employment

'''

markdown_text_Tab3_8 = '''
#### Family

'''

markdown_text_Tab3_9 = '''
#### Health

'''

markdown_text_Tab3_10 = '''
#### Language

'''

markdown_text_Tab3_11 = '''
#### Development

'''

markdown_text_Tab3_12 = '''
#### Business

'''

markdown_text_Tab3_13 = '''
#### Volunteer

'''

markdown_text_Tab3_14 = '''
#### Finance

'''

markdown_text_Tab3_15 = '''
#### Indigenous

'''

markdown_text_Tab3_16 = '''
#### Features Class on Different Aggregation Level

'''

markdown_text_Tab3_17 = '''
#### Complete Features Class on Different Aggregation Level

'''


markdown_text_Tab3_18 = '''
#### Sentiment Polarity

'''

markdown_text_Tab3_19 = '''
#### Most Common Topic

'''

cluster_size = 3 
cluster_list =[]
for i in range(cluster_size):
    cluster_list.append('cluster_' + str(i+1))


PAGE_SIZE = 5

image_directory = 'images/'
header_file = 'header.png'
static_file_name = 'static.png'

header_filename =  image_directory + header_file
encoded_header = base64.b64encode(open(header_filename, 'rb').read())        


app.layout = html.Div([
    
                html.Img(
                    src = 'data:image/png;base64,{}'.format(encoded_header.decode()),  
                    style={'width': '35%', 'display': 'inline-block', 'margin-top':10,'margin-bottom':10,'margin-left':10,'margin-right':10}
                ),
    

                dcc.Tabs(id="tabs", children=[
                    dcc.Tab(label='Recommendation', children=[
                        
                        html.Div(
                            className="row",
                            children=[
                                
                                html.Div(
                                    [
                                 
                                        dcc.Markdown(children=markdown_text_Tab1_1),    


                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""Your favorite state""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),

                                                dcc.Dropdown(
                                                    id='state-dropdown',
                                                    options=[{'label': k, 'value': k} for k in sa4_name.keys()],
                                                    value=list(sa4_name.keys())[0],
                                                    style=dict(
                                                        width='60%',
                                                        display='inline-block'
                                                    )
                                                )

                                            ],
                                            style=dict(display='flex')
                                        ),

                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""Your favorite area""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),


                                                dcc.Dropdown(
                                                    id='sa4-dropdown',
                                                    style=dict(
                                                        width='60%',
                                                        display='inline-block'                                                          
                                                    )
                                                )

                                            ],
                                            style=dict(display='flex')
                                        ),

                                        html.Div(id='intermediate-value', style={'display': 'none'}),        
                                        
                                        html.Hr(),                                     
                                                                           
                                        
                                        dcc.Markdown(children=markdown_text_Tab1_2), 
                                                                         
                                        
                                        dcc.Checklist(                                           
                                            id='feature-checklist',
                                            options=[{'label': k, 'value': k} for k in feature_rename],
                                            value=feature_rename,
                                            labelStyle={'display': 'inline-block'}

                                        ),  
                                        
                                        
                                        ### start of the feature importance
                                        
                                        html.Div( 
                                            [  
                                             html.Div([

                                                html.Div(
                                                    [

                                                     html.Div(id='slider-output1', style={'margin-top': 10}),

                                                     dcc.Slider(
                                                        id='feature1',
                                                     ),

                                                    html.Br(),


                                                    ], className="three columns"

                                                ),                      


                                                html.Div(
                                                    [

                                                     html.Div(id='slider-output2', style={'margin-top': 10}),


                                                     dcc.Slider(
                                                        id='feature2',
                                                     ),

                                                    html.Br(),


                                                    ], className="three columns"

                                                ),     


                                                html.Div(
                                                    [
                                                     html.Div(id='slider-output3', style={'margin-top': 10}),

                                                     dcc.Slider(
                                                        id='feature3',
                                                     ),

                                                    html.Br(),


                                                    ], className="three columns"

                                                ),                      

                                                html.Div(
                                                    [
                                                     html.Div(id='slider-output4', style={'margin-top': 10}),
                                                        
                                                     dcc.Slider(
                                                        id='feature4',
                                                     ),

                                                    html.Br(),


                                                    ], className="three columns"

                                                )                                                         


                                            ],  className="rows",   ## end of first four
                                                style={
                                                        'margin-top':10,'margin-bottom':10,
                                                        'margin-left':20,'margin-right':20
                                                       }                                                 
                                            ),

                                            html.Hr(), 

                                            html.Div([

                                                html.Div(
                                                    [
                                                     html.Div(id='slider-output5', style={'margin-top': 20}),
                                                        
                                                     dcc.Slider(
                                                        id='feature5',
                                                     ),

                                                    html.Br(),

                                                    ], className="three columns"

                                                ),     

                                                html.Div(
                                                    [
                                                     html.Div(id='slider-output6', style={'margin-top': 20}),
                                                        
                                                     dcc.Slider(
                                                        id='feature6',
                                                     ),


                                                    html.Br(),


                                                    ], className="three columns"

                                                ),                      


                                                html.Div(
                                                    [
                                                     html.Div(id='slider-output7', style={'margin-top': 20}),
                                                        
                                                     dcc.Slider(
                                                        id='feature7',
                                                     ),


                                                    html.Br(),

                                                    ], className="three columns"

                                                ),                      

                                                html.Div(
                                                    [
                                                     html.Div(id='slider-output8', style={'margin-top': 20}),
                                                        
                                                     dcc.Slider(
                                                        id='feature8',
                                                     ),


                                                    html.Br(),


                                                    ], className="three columns"

                                                ),

                                                html.Hr(), 


                                            ],  className="rows",  ## end of second four
                                                style={
                                                        'margin-top':10,'margin-bottom':10,
                                                        'margin-left':20,'margin-right':20
                                                       }

                                            ),                                                                                  
                                            
                                            
                                        ],style={
                                                'height': 180,
                                                'borderWidth' : 1,
                                                'borderStyle' : 'dashed',
                                                'borderRadius': 10,
                                                'textAlign': 'center',
                                                'margin': 10,
                                                'background-color': '#09385A',
                                                'color': '#FFFFFF'
                                            },
                                        
                                        ), ## end of feature preferences

                                        html.Hr(), 
                                        
                                        html.Div(id='display-selected-values'),
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_show',
                                                fixed_columns={ 'headers': True, 'data': 2 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
                                                
                                                style_table={
                                                    'maxHeight': '60ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
#                                                     'textAlign': 'center',
#                                                     'height': '60px',
#                                                     'minWidth': '100%',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                
                                               style_cell_conditional=[
                                                    {
                                                        'if': {'column_id': 'Areas Name'},
                                                        'textAlign': 'left'
                                                    },
                                                ],
                                                
                                                # style header
                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },
                                                # style filter
                                                # style data
                                                style_data_conditional=[
                                                    {
                                                        # stripped rows
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        # highlight one row
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    }
                                                ],

                                            
                                            )                                              
                                        ),
                                        
                                        html.Hr(),   
                                        
                                        dcc.Markdown(children=markdown_text_Tab1_6),
                                        
                                        dcc.Graph(
                                            id='box-plot'
                                        ),

                                        
                                        html.Hr(),   

                                        dcc.Markdown(children=markdown_text_Tab1_7),
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_categorical_show',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
#                                                     'textAlign': 'center',
#                                                     'height': '60px',
#                                                     'minWidth': '100%',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },


                                               style_cell_conditional=[
                                                    {
                                                        'if': {'column_id': 'Areas Name'},
                                                        'textAlign': 'left'
                                                    },
                                                ],

                                                # style header
                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },
                                                # style filter
                                                # style data
                                                style_data_conditional=[
                                                    {
                                                        # stripped rows
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        # highlight one row
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    }
                                                ],


                                            )                                              
                                        ),
                                        
                                        html.Div(id='intermediate-value2', style={'display': 'none'}),        

                                        
                                        html.Hr(),   
                                        
                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""Select Cluster""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),


#                                                 dcc.Dropdown(
#                                                     id='cluster-dropdown',
#                                                     options=[{'label': k, 'value': k} for k in cluster_list],
# #                                                     value=list(sa4_name.keys())[0],
                                                
#                                                     style=dict(
#                                                         width='60%',
#                                                         display='inline-block'                                                          
#                                                     )
#                                                 )
                                                
                                                                                                                                                                          
                                                dcc.RadioItems(
                                                    id='cluster-dropdown',
                                                    options=[{'label': k, 'value': k} for k in cluster_list],
                                                    labelStyle={'display': 'inline-block'}
                                                )                                         
       

                                            ],
                                            style=dict(display='flex')
                                        ),
                                        
                                        
                                        
                              
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_cluster_selected_show',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
#                                                     'textAlign': 'center',
#                                                     'height': '60px',
#                                                     'minWidth': '100%',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },


                                               style_cell_conditional=[
                                                    {
                                                        'if': {'column_id': 'Areas Name'},
                                                        'textAlign': 'left'
                                                    },
                                                ],

                                                # style header
                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },
                                                # style filter
                                                # style data
                                                style_data_conditional=[
                                                    {
                                                        # stripped rows
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        # highlight one row
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    }
                                                ],


                                            )                                              
                                        ),
                                                                                
                                        html.Hr(),   
  
                                        

                                    ],
                                    className='six columns'
                                ),
                               
                            
                                html.Div(                                
                                    [

                                        html.Div(
                                            [
                                                dcc.Graph(id='map_live0',
                                                          config={
                                                                    'scrollZoom': True
                                                                 })
                                                
                                            ]
                                        ),
                                        
                                        
                                        dcc.Markdown(children=markdown_text_Tab1_4),    
                                        
                                        html.Img(
                                            id='display-map',
                                            style={'width': '100%', 'display': 'inline-block', 'margin-top':0,'margin-bottom':0,
                                                   'margin-left':0,'margin-right':0}
                                        ),
                                        
                                                                                
                                           
                                        html.Hr(),   

                                        
                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""Select area""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),


                                                dcc.Dropdown(
                                                    id='sa2-dropdown',
                                                    style=dict(
                                                        width='60%',
                                                        display='inline-block'                                                          
                                                    )
                                                )

                                            ],
                                            style=dict(display='flex')
                                        ),
                                        
                                        
                                        dcc.Markdown(children=markdown_text_Tab1_5),    
                                        
                                        html.Div([
                                            dcc.Graph(
                                                id='deviation',
                                            )]
                                        ),
                                        
                                        
#                                         dcc.Markdown(children=markdown_text_Tab1_9),    
                                        
#                                         html.Div([
#                                             dcc.Graph(
#                                                 id='polar-bar',
#                                             )
#                                         ]),                                        
                                                                                
                                        
                                        
                                        
                                        html.Hr(),   

                                        dcc.Markdown(children=markdown_text_Tab1_8),
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_cluster_show',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
#                                                     'textAlign': 'center',
#                                                     'height': '60px',
#                                                     'minWidth': '100%',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },


                                               style_cell_conditional=[
                                                    {
                                                        'if': {'column_id': 'Areas Name'},
                                                        'textAlign': 'left'
                                                    },
                                                ],

                                                # style header
                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },
                                                # style filter
                                                # style data
                                                style_data_conditional=[
                                                    {
                                                        # stripped rows
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        # highlight one row
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    }
                                                ],


                                            )                                              
                                        ),                                                   
                                        
                                        
                                        html.Hr(),   


                                        dcc.Markdown(children=markdown_text_Tab1_10),
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_cluster_aussie_show',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
#                                                     'textAlign': 'center',
#                                                     'height': '60px',
#                                                     'minWidth': '100%',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },


                                               style_cell_conditional=[
                                                    {
                                                        'if': {'column_id': 'Areas Name'},
                                                        'textAlign': 'left'
                                                    },
                                                ],

                                                # style header
                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },
                                                # style filter
                                                # style data
                                                style_data_conditional=[
                                                    {
                                                        # stripped rows
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        # highlight one row
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    }
                                                ],


                                            )                                              
                                        ),                                                   
                                        
                                        html.Hr(),   

                                    ],
                                    className="five columns"
                                ),

                            ]),    ## end of children of div                    
                        
                    ]),
                    
                    dcc.Tab(label='Comparison', children=[
                                               
                        ### Comparison Page

                        dcc.Markdown(children=markdown_text_Tab2_1),                     

                        html.Div([
                        *[
                            html.Div([
                                dcc.Dropdown(
                                     id= 'feature-dropdown' + str(i),
                                    options= [{'label': k, 'value': k} for k in features_dict.keys()],
                                    value=list(features_dict.keys())[i],
                                ),
                            ], style=dict(width='25%', display='inline-block'))
                        for i in range(3)]
                        ]),
                        
                        
                        html.Div([
                        *[
                            html.Div([
                                dcc.Dropdown(
                                    id = 'feature1-dropdown'),
                            ], 
                            style=dict(width='25%', display='inline-block')),
                            
                            html.Div([
                                dcc.Dropdown(
                                    id = 'feature2-dropdown'),
                            ], 
                            style=dict(width='25%', display='inline-block')),
                            
                            html.Div([
                                dcc.Dropdown(
                                    id = 'feature3-dropdown'),
                            ], 
                            style=dict(width='25%', display='inline-block'))                            
                        ]
                        ]),
                                          
                        html.Div([
                        *[
                            html.Div(id = 'display-selected-features1'), 
                            html.Div(id = 'display-selected-features2'), 
                            html.Div(id = 'display-selected-features3'),                          
                        ]
                        ]),
                        
                        
                        html.Hr(),   
                        
                        html.Div(
                            className="row",
                            children=[
                                html.Div(
                                    dash_table.DataTable(
                                        id='table-paging-with-graph',
                                        columns=[
                                            {"name": i, "id": i} for i in sorted(df.columns)
                                        ],
                                        page_current=0,
                                        page_size=20,
                                        page_action='custom',

                                        filter_action='custom',
                                        filter_query='',

                                        sort_action='custom',
                                        sort_mode='multi',
                                        sort_by=[]
                                    ),
                                    style={'height': 750, 'overflowY': 'scroll'},
                                    className='six columns'
                                ),
                                html.Div(
                                    id='table-paging-with-graph-container',
                                    className="five columns"
                                )
                            ])

                        
                        
                        
                    ]),
                    
                    dcc.Tab(label='City Profile', children=[    
                        
                        
                                                 

                        html.Div(
                            className="row",
                            children=[
                                html.Div(
                                    [

                                        html.Hr(),   

                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""State""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),

                                                dcc.Dropdown(
                                                    id='tab3_state-dropdown',
                                                    options=[{'label': k, 'value': k} for k in sa4_name2.keys()],
                                                    value=list(sa4_name2.keys())[0],
                                                    style=dict(
                                                        width='60%',
                                                        display='inline-block'
                                                    )
                                                )

                                            ],
                                            style=dict(display='flex')
                                        ),

                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""Group""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),


                                                dcc.Dropdown(
                                                    id='tab3_sa4-dropdown',
                                                    style=dict(
                                                        width='60%',
                                                        display='inline-block'                                                          
                                                    )
                                                )

                                            ],
                                            style=dict(display='flex')
                                        ),


                                        html.Div(
                                            [
                                                html.Div(
                                                    [
                                                        html.H6("""Area""",
                                                                style={'margin-right': '2em'})
                                                    ],
                                                ),


                                                dcc.Dropdown(
                                                    id='tab3_sa2-dropdown',
                                                    style=dict(
                                                        width='60%',
                                                        display='inline-block'                                                          
                                                    )
                                                )

                                            ],
                                            style=dict(display='flex')
                                        ),

                                        html.Div(
                                            [
                                                dcc.Graph(id='map_live',
                                                          config={
                                                                    'scrollZoom': True
                                                                 })                                           
                                            ]
                                           
                                        ),
                                        
                                        html.Hr(),   
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_16),  
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_cat_all',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                ],


                                            )                                              
                                        ),                                         

                                        
                                        html.Hr(),   
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_17),  
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_cat_complete',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                ],


                                            )                                              
                                        ),      
                                        
                                        html.Hr(),   
                                              
                                        dcc.Markdown(children=markdown_text_Tab3_18),  

                                        
                                        html.Div(id='display-sentiment'),
                                        
                                        html.Hr(),   
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_19),  

                                                                          
                                        html.Div([
                                            dcc.Graph(
                                                id='topic',
                                            )]
                                        ),      
                                        
                                        
                                        html.Hr(),   

                                    ],

                                    className='six columns'
                                ),
                                
                                
                                

                                
                                html.Div(                                
                                    [
                                        dcc.Markdown(children=markdown_text_Tab3_1),  
                                        
                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show1',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                ],


                                            )                                              
                                        ),  
                                        
                                        
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_2),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show2',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                          
 
                                        
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_3),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show3',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ), 
                                        
                                                                                
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_4),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show4',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ), 
                                        
                                        dcc.Markdown(children=markdown_text_Tab3_5),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show5',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ), 

                                        dcc.Markdown(children=markdown_text_Tab3_6),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show6',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         

                                        dcc.Markdown(children=markdown_text_Tab3_7),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show7',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         

                                        dcc.Markdown(children=markdown_text_Tab3_8),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show8',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ), 


                                        dcc.Markdown(children=markdown_text_Tab3_9),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show9',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         


                                        dcc.Markdown(children=markdown_text_Tab3_10),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show10',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         

                                        

                                        dcc.Markdown(children=markdown_text_Tab3_11),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show11',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),   
                                        

                                        dcc.Markdown(children=markdown_text_Tab3_12),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show12',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         

                                        dcc.Markdown(children=markdown_text_Tab3_13),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show13',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),  

                                        dcc.Markdown(children=markdown_text_Tab3_14),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show14',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         
                                        

                                        dcc.Markdown(children=markdown_text_Tab3_15),    

                                        html.Div(
                                            dash_table.DataTable(
                                                id='data_selected_show15',
                                                fixed_columns={ 'headers': True, 'data': 1 },
                                                fixed_rows={ 'headers': True, 'data': 0 },
                                                editable=True,
                                                sort_action="native",
                                                sort_mode="multi",
#                                                 filter_action="native",
#                                                 page_action='native',

                                                style_table={
                                                    'maxHeight': '40ex',
                                                    'overflowY': 'scroll',
                                                    'width': '100%',
                                                    'minWidth': '100%',
                                                },
                                                # style cell
                                                style_cell={
                                                    'fontFamily': 'Helvetica',
                                                    'padding': '2px 50px',
                                                    'whiteSpace': 'inherit',
                                                    'overflow': 'hidden',
                                                    'textOverflow': 'ellipsis',
                                                },

                                                style_header={
                                                    'fontWeight': 'bold',
                                                    'backgroundColor': 'white',
                                                },

                                                style_data_conditional=[
                                                    {
                                                        'if': {'row_index': 'odd'},
                                                        'backgroundColor': 'rgb(248, 248, 248)'
                                                    },
                                                    {
                                                        'if': {'row_index': 0},
                                                        "backgroundColor": "#09385A",
                                                        'color': 'white'
                                                    },
                                                    {
                                                        'if': {'column_id': 'Value', 'filter_query': '{{{0}}} < 0.0'.format('Value')},
                                                        'color': 'red',
                                                    }
                                                    
                                                ],


                                            )                                              
                                        ),                                         
                                    ],
                                    className="five columns"
                                ),

                        ]),
                        
                    ]),
                    
#                     dcc.Tab(label='Analysis', children=[    
                        
#                     ])
                ])
            ])
                    
#---------------------------------------------------------------------------------
# AREA SELECTION
#---------------------------------------------------------------------------------


@app.callback(
    Output('sa4-dropdown', 'options'),
    [Input('state-dropdown', 'value')])
def set_cities_options1(selected_state):
    return [{'label': i, 'value': i} for i in sa4_name[selected_state]]


@app.callback(
    Output('sa4-dropdown', 'value'),
    [Input('sa4-dropdown', 'options')])
def set_cities_value1(available_options):
    return available_options[0]['value']


@app.callback(
    Output('display-selected-values', 'children'),
    [Input('state-dropdown', 'value'),
     Input('sa4-dropdown', 'value')])
def set_display_children1(selected_state, selected_sa4):
    if selected_sa4 != 'Any':
        mark_text1 = f'''#### Recommended areas in {selected_state}'''
        mark_text2 = f'''##### - {selected_sa4} -'''        
        return dcc.Markdown(children=mark_text1),dcc.Markdown(children=mark_text2)    
    else:
        mark_text = f'''#### Recommended areas in {selected_state}'''
        return dcc.Markdown(children=mark_text)  
                                               

@app.callback(
        Output('intermediate-value', 'children'), 
        [Input('state-dropdown', 'value'),
         Input('sa4-dropdown', 'value')])
def intermediate_data(selected_state, selected_sa4):
    df_results = get_data_results(selected_state, selected_sa4)
    feature_border = get_feature_border(df_results)
    datasets = {
         'df_results': df_results.to_json(orient='split', date_format='iso'),
         'feature_border': feature_border,
    }
    return json.dumps(datasets)


def get_data_results(selected_state, selected_sa4):    
    dt_temp = data_selected.copy()   
    if selected_sa4 != 'Any':
        dt_temp = dt_temp[(dt_temp.STATE_NAME_2016 == selected_state) & (dt_temp.SA4_NAME_2016 == selected_sa4)]
    elif selected_state != 'Australia' and selected_sa4 == 'Any':
        dt_temp = dt_temp[dt_temp.STATE_NAME_2016 == selected_state]               
    dt_temp = dt_temp[column_name]
    for i in range(len(column_name)):
        dt_temp = dt_temp.rename(columns={column_name[i]:column_rename[i]})      
    return dt_temp    
        
def get_feature_border(data):     
    feature_border = {}
    for feature in feature_rename:
        feature_border[feature+'_min'] = data[feature].min()
        feature_border[feature+'_max'] = data[feature].max()
        feature_border[feature+'_mean'] = data[feature].mean()
        feature_border[feature+'_std'] = data[feature].std()  
        feature_border[feature+'_value'] = data[feature].mean()        
    return feature_border


@app.callback(
        [Output('feature1', 'min'),
         Output('feature1', 'max'),
         Output('feature1', 'value'),
         Output('feature1', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
#     df_results = pd.read_json(datasets['df_results'], orient='split')
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[0]+'_min'],
            feature_border[feature_rename[0]+'_max'], 
            feature_border[feature_rename[0]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[0]+'_min'],feature_border[feature_rename[0]+'_max'],5)}]
    

@app.callback(
        [Output('feature2', 'min'),
         Output('feature2', 'max'),
         Output('feature2', 'value'),
         Output('feature2', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[1]+'_min'],
            feature_border[feature_rename[1]+'_max'], 
            feature_border[feature_rename[1]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[1]+'_min'],feature_border[feature_rename[1]+'_max'],5)}]
    

@app.callback(
        [Output('feature3', 'min'),
         Output('feature3', 'max'),
         Output('feature3', 'value'),
         Output('feature3', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[2]+'_min'],
            feature_border[feature_rename[2]+'_max'], 
            feature_border[feature_rename[2]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[2]+'_min'],feature_border[feature_rename[2]+'_max'],5)}]
        

@app.callback(
        [Output('feature4', 'min'),
         Output('feature4', 'max'),
         Output('feature4', 'value'),
         Output('feature4', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[3]+'_min'],
            feature_border[feature_rename[3]+'_max'], 
            feature_border[feature_rename[3]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[3]+'_min'],feature_border[feature_rename[3]+'_max'],5)}]


@app.callback(
        [Output('feature5', 'min'),
         Output('feature5', 'max'),
         Output('feature5', 'value'),
         Output('feature5', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[4]+'_min'],
            feature_border[feature_rename[4]+'_max'], 
            feature_border[feature_rename[4]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[4]+'_min'],feature_border[feature_rename[4]+'_max'],5)}]
     
    
@app.callback(
        [Output('feature6', 'min'),
         Output('feature6', 'max'),
         Output('feature6', 'value'),
         Output('feature6', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[5]+'_min'],
            feature_border[feature_rename[5]+'_max'], 
            feature_border[feature_rename[5]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[5]+'_min'],feature_border[feature_rename[5]+'_max'],5)}]


@app.callback(
        [Output('feature7', 'min'),
         Output('feature7', 'max'),
         Output('feature7', 'value'),
         Output('feature7', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[6]+'_min'],
            feature_border[feature_rename[6]+'_max'], 
            feature_border[feature_rename[6]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[6]+'_min'],feature_border[feature_rename[6]+'_max'],5)}]
        
        
@app.callback(
        [Output('feature8', 'min'),
         Output('feature8', 'max'),
         Output('feature8', 'value'),
         Output('feature8', 'marks')], 
        [Input('intermediate-value', 'children')])
def update_feature1(json_data):
    datasets = json.loads(json_data)
    feature_border = datasets['feature_border']
    return [feature_border[feature_rename[7]+'_min'],
            feature_border[feature_rename[7]+'_max'], 
            feature_border[feature_rename[7]+'_value'],
            {i: '{:.0f}'.format(i) for i in np.linspace(feature_border[feature_rename[7]+'_min'],feature_border[feature_rename[7]+'_max'],5)}]
               
                                                        
                
#---------------------------------------------------------------------------------
# FEATURES SELECTION
#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('slider-output1', 'children'),
    [dash.dependencies.Input('feature1', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[0]+'_value'] = value
        output_value = feature_border[feature_rename[0]+'_value']
    else:
        output_value = 0.0
    return feature_rename[0].upper() + ': ${:.0f}0,000'.format(output_value)


#---------------------------------------------------------------------------------

@app.callback(
    dash.dependencies.Output('slider-output2', 'children'),
    [dash.dependencies.Input('feature2', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[1]+'_value'] = value
        output_value = feature_border[feature_rename[1]+'_value']
    else:
        output_value = 0.0
    return feature_rename[1].upper() + ': ${:.0f},000'.format(output_value)

#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('slider-output3', 'children'),
    [dash.dependencies.Input('feature3', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[2]+'_value'] = value
        output_value = feature_border[feature_rename[2]+'_value']
    else:
        output_value = 0.0
    return feature_rename[2].upper() + ': {:.0f}0'.format(output_value)


#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('slider-output4', 'children'),
    [dash.dependencies.Input('feature4', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[3]+'_value'] = value
        output_value = feature_border[feature_rename[3]+'_value']
    else:
        output_value = 0.0
    return feature_rename[3].upper() + ': {:.0f}%'.format(output_value)


#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('slider-output5', 'children'),
    [dash.dependencies.Input('feature5', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[4]+'_value'] = value
        output_value = feature_border[feature_rename[4]+'_value']
    else:
        output_value = 0.0
    return feature_rename[4].upper() + ': {:.0f}'.format(output_value)


#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('slider-output6', 'children'),
    [dash.dependencies.Input('feature6', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[5]+'_value'] = value
        output_value = feature_border[feature_rename[5]+'_value']
    else:
        output_value = 0.0
    return feature_rename[5].upper() + ': {:.0f},000'.format(output_value)


#---------------------------------------------------------------------------------



@app.callback(
    dash.dependencies.Output('slider-output7', 'children'),
    [dash.dependencies.Input('feature7', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[6]+'_value'] = value
        output_value = feature_border[feature_rename[6]+'_value']
    else:
        output_value = 0.0
    return feature_rename[6].upper() + ': {:.0f}%'.format(output_value)


#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('slider-output8', 'children'),
    [dash.dependencies.Input('feature8', 'value'),
     dash.dependencies.Input('intermediate-value', 'children')
    ])
def update_output(value,json_data):
    if value is not None:
        datasets = json.loads(json_data)
        feature_border = datasets['feature_border']
        feature_border[feature_rename[7]+'_value'] = value
        output_value = feature_border[feature_rename[7]+'_value']
    else:
        output_value = 0.0
    return feature_rename[7].upper() + ': {:.0f}0'.format(output_value)



#---------------------------------------------------------------------------------
# SHOW TABLE RECOMMENDED AREAS
#---------------------------------------------------------------------------------

        
@app.callback(
    [dash.dependencies.Output('data_show', 'data'),
     dash.dependencies.Output('data_show', 'columns')
    ],
    [     
     dash.dependencies.Input('feature1', 'value'),  
     dash.dependencies.Input('feature2', 'value'),  
     dash.dependencies.Input('feature3', 'value'),  
     dash.dependencies.Input('feature4', 'value'),  
     dash.dependencies.Input('feature5', 'value'),  
     dash.dependencies.Input('feature6', 'value'),  
     dash.dependencies.Input('feature7', 'value'),  
     dash.dependencies.Input('feature8', 'value'),
     dash.dependencies.Input('intermediate-value', 'children'),
     dash.dependencies.Input('feature-checklist', 'value') 
    ])
def update_table(value1,value2,value3,value4,value5,value6,value7,value8,json_data,feature_list):
    
    if value1 is not None and value2 is not None and value3 is not None and value4 is not None and \
       value5 is not None and value6 is not None and value7 is not None and value8 is not None:      
        datasets = json.loads(json_data)
        df_results = pd.read_json(datasets['df_results'], orient='split')
        feature_border = datasets['feature_border']          
        feature_border[feature_rename[0]+'_value'] = value1
        feature_border[feature_rename[1]+'_value'] = value2
        feature_border[feature_rename[2]+'_value'] = value3
        feature_border[feature_rename[3]+'_value'] = value4
        feature_border[feature_rename[4]+'_value'] = value5
        feature_border[feature_rename[5]+'_value'] = value6
        feature_border[feature_rename[6]+'_value'] = value7
        feature_border[feature_rename[7]+'_value'] = value8       
        data = get_data_score(df_results,feature_border,feature_list)
        columns = [{'id': c, 'name': c} for c in data.columns]    
    else:
        data = pd.DataFrame()
        columns = []
    return [data.to_dict('records'),columns]


def get_data_score(df_results,feature_border,feature_list):    
    df_std = df_results.copy()
    feature_list_all = ['Areas Name'] + feature_list
    df_std = df_std[feature_list_all]
    feature_expectation = {}
    for feature in feature_list:  
        feature_expectation[feature] = (feature_border[feature+'_value'] - feature_border[feature+'_min']) / (feature_border[feature+'_max'] - feature_border[feature+'_min'])
        df_std[feature] = df_std.apply(lambda x: (x[feature] - feature_border[feature+'_min']) / (feature_border[feature+'_max'] - feature_border[feature+'_min']),axis=1)

    df_score = df_std.copy()
        
    for feature in feature_list:  
        df_score[feature] = df_score.apply(lambda x: abs(x[feature] - feature_expectation[feature]),axis=1)

    df_score['Score'] = (1 - (df_score.iloc[:,1:].sum(axis=1)/len(feature_list)))*100
    
    
    df_score['Rank'] = df_score['Score'].rank(ascending=False) 

    df_score.sort_values('Rank', inplace = True) 

    df_show = df_score[['Areas Name','Rank','Score']]
    
    df_results_act = df_results.copy()
    
    
    df_results_act['Family'] = df_results_act.apply(lambda x: x.Family*10, axis=1) 
    df_results_act['Population'] = df_results_act.apply(lambda x: x.Population*10 , axis=1)
    df_results_act['Housevalue'] = df_results_act.apply(lambda x: x.Housevalue*10000 , axis=1)
    df_results_act['Income'] = df_results_act.apply(lambda x: x.Income*1000 , axis=1)
    df_results_act['Employment'] = df_results_act.apply(lambda x: x.Employment*1000 , axis=1)
    
    df_show = pd.merge(df_show, df_results_act, on='Areas Name')

    for feature in ['Score'] + feature_rename:  
        df_show[feature] = df_show.apply(lambda x: round(x[feature],1) ,axis=1)

    return df_show


 

#---------------------------------------------------------------------------------
# Box-Plot
#---------------------------------------------------------------------------------



@app.callback(Output('box-plot', 'figure'),
              [Input('data_show', 'data')])
def update_graph(data):
    data = pd.DataFrame.from_dict(data)  
    if not data.empty:            
        
        score_features = data.iloc[:,3:]
        trace = {}

        for feature in list(score_features.columns):  
            feature_min = score_features[feature].min()
            feature_max = score_features[feature].max()
            score_features[feature] = score_features.apply(lambda x: (x[feature] - feature_min) / (feature_max - feature_min),axis=1)  
            trace[feature] = go.Box(y=list(score_features[feature]), name=feature,)

        data = list(trace.values())
        return {
            'data': data,
            'layout': go.Layout(
                title='Box Plot'
            )
        }
    else:
        return {
            'data': [],
            'layout': go.Layout(
                title='Box Plot'
            )
        }        





#---------------------------------------------------------------------------------
# Data Categorical
#---------------------------------------------------------------------------------


@app.callback(
    [dash.dependencies.Output('data_categorical_show', 'data'),
     dash.dependencies.Output('data_categorical_show', 'columns')
    ],
    [     
     dash.dependencies.Input('data_show', 'data')
    ])

def update_table2(data):
    data = pd.DataFrame.from_dict(data)  
    if not data.empty:
        score_features = data.iloc[:,2:]

        df_std = pd.DataFrame(StandardScaler().fit_transform(score_features), columns = score_features.columns)

        mean_list = df_std.mean(skipna = True)
        stdev_list = df_std.std(skipna = True)

        middle_upper = mean_list  +  stdev_list
        middle_lower = mean_list  -  stdev_list
        list_key = mean_list.keys()

        new_df = pd.DataFrame()

        new_df['Areas Name'] = data.iloc[:,0]

        for key in list_key:
            upper_bound = middle_upper[key]
            lower_bound = middle_lower[key]
            key_values = df_std[key].values
            new_values = []
            for i in range(len(key_values)):
                try:
                    if key_values[i] > upper_bound:
                        new_values.append('High') 
                    elif key_values[i] < lower_bound:
                        new_values.append('Low') 
                    else:
                        new_values.append('Medium') 
                except:
                    continue
            new_df[key] = new_values    

        data_categorical = new_df.copy()
        columns = [{'id': c, 'name': c} for c in data_categorical.columns]   
        
    else:
        data_categorical = pd.DataFrame()
        columns = []        
   

    return [data_categorical.to_dict('records'),columns]



#---------------------------------------------------------------------------------
# Cluster Analysis
#---------------------------------------------------------------------------------

@app.callback(
    [dash.dependencies.Output('data_cluster_show', 'data'),
     dash.dependencies.Output('data_cluster_show', 'columns'),
     dash.dependencies.Output('intermediate-value2', 'children')     
    ],
    [     
     dash.dependencies.Input('data_categorical_show', 'data')
    ])
def update_table3(data):
    data = pd.DataFrame.from_dict(data)  
    if not data.empty:
        df_cat = data.iloc[:,1:]              
        km = KModes(n_clusters=cluster_size, init='Huang', n_init=5, verbose=1)
        clusters = km.fit_predict(df_cat)  
        
        df_results = pd.DataFrame({'Feature':  list(df_cat.columns) })        
        
        for i in range(len(km.cluster_centroids_)):
            cluster_name = 'cluster_' + str(i+1)
            df_results[cluster_name] = km.cluster_centroids_[i]         
        
        columns1 = [{'id': c, 'name': c} for c in df_results.columns]   
        
        df_cat['cluster'] = clusters + 1
        df_cat['Areas Name'] = data.iloc[:,0]
                
        datasets = {
             'df_cluster': df_cat.to_json(orient='split', date_format='iso')
        }        
            
        
    else:
        df_results = pd.DataFrame()
        columns1 = []   
        datasets = {}

    return [df_results.to_dict('records'),columns1,json.dumps(datasets)]


@app.callback(
    [
     dash.dependencies.Output('data_cluster_selected_show', 'data'),     
     dash.dependencies.Output('data_cluster_selected_show', 'columns')          
    ],
    [     
     dash.dependencies.Input('intermediate-value2', 'children'),
     dash.dependencies.Input('cluster-dropdown', 'value')
    ])
def update_table4(json_data,value):
    if value is not None:
        datasets = json.loads(json_data)
        df_cluster = pd.read_json(datasets['df_cluster'], orient='split')
        cluster_chosen = int(value.split("_")[1])
        areas_in_cluster = df_cluster[df_cluster.cluster == cluster_chosen][['Areas Name','Score']+feature_rename]      
        columns2 = [{'id': c, 'name': c} for c in areas_in_cluster.columns]   
        
    else: 
        areas_in_cluster = pd.DataFrame()
        columns2 = []           

    return [areas_in_cluster.to_dict('records'),columns2]


@app.callback(
    [
     dash.dependencies.Output('data_cluster_aussie_show', 'data'),     
     dash.dependencies.Output('data_cluster_aussie_show', 'columns')          
    ],
    [     
     dash.dependencies.Input('data_cluster_selected_show', 'data')
    ])
def update_table5(data):
    data = pd.DataFrame.from_dict(data)  
    if not data.empty:
        sa2_selected_cluster = data.iloc[:,:1]
        
        data_cluster_desc = sa2_selected_cluster.set_index('Areas Name').join(df_kmodes_all.set_index('Areas Name'), how = 'inner').reset_index()
        columns = [{'id': c, 'name': c} for c in data_cluster_desc.columns]   
        
    else: 
        data_cluster_desc = pd.DataFrame()
        columns = []           

    return [data_cluster_desc.to_dict('records'),columns]

    
    

#---------------------------------------------------------------------------------
# VISUALISE MAP
#---------------------------------------------------------------------------------


@app.callback(
    Output('display-map', 'src'),
    [     
     Input('data_show', 'data'),
     Input('state-dropdown', 'value'),
     Input('sa4-dropdown', 'value'),
    ])
def update_image_src(data,selected_state,selected_sa4):
    data_score = pd.DataFrame.from_dict(data)   
    if not data_score.empty:
        data_score = data_score[['Areas Name','Score']]
        if selected_sa4 == 'Any':
            title = selected_state
        else:
            title = selected_state + " - " + selected_sa4
            
        data_score = data_score.set_index('Areas Name').join(sa2_code_name.set_index('SA2_NAME_2016'), how = 'inner')
        data_score = data_score[['SA2_CODE','Score']]
        data_score['SA2_CODE'] = data_score['SA2_CODE'].astype(str)
        map_path = map(data_score,title)    
        image_filename =  image_directory + map_path
        encoded_image = base64.b64encode(open(image_filename, 'rb').read())
        
        return 'data:image/png;base64,{}'.format(encoded_image.decode())
    else:
        image_filename =  image_directory + 'default.png'
        encoded_image = base64.b64encode(open(image_filename, 'rb').read())
        return 'data:image/png;base64,{}'.format(encoded_image.decode())
                
def map(data_score,title):
    merged = map_df.set_index('SA2_MAIN16').join(data_score.set_index('SA2_CODE'), how = 'inner')
#     merged['Score'] = (merged['Score']/max(merged['Score']))*100
    vmin = merged['Score'].min()
    vmax = merged['Score'].max()

    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.axis('off')
    
    merged.plot(column='Score', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')
    
    ax.set_title(title, fontdict={'fontsize': '15', 'fontweight' : '3', 'horizontalalignment': 'center'})
    
    sm = plt.cm.ScalarMappable(cmap='Blues',norm=plt.Normalize(vmin,vmax))
    sm._A = []
    
    cbar = fig.colorbar(sm, shrink=0.5)
    cbar.ax.tick_params(labelsize=10) 

    fig.savefig(image_directory + static_file_name, dpi=200,bbox_inches='tight')    
    plt.close(fig)
    
    return static_file_name


#---------------------------------------------------------------------------------
# VISUALISE MAP LIVE
#---------------------------------------------------------------------------------


@app.callback(
    dash.dependencies.Output('map_live0', 'figure'),
    [dash.dependencies.Input('state-dropdown', 'value'),
     dash.dependencies.Input('sa4-dropdown', 'value')])
def display_map1(selected_state,selected_sa4):
    if selected_state is not None and selected_sa4 is not None:
        if selected_state == 'Australia' and selected_sa4 == 'Any':
            lat = list(sa2_spatial['lat'])  
            lon = list(sa2_spatial['lon'])
            center_lat =  -25.2743988
            center_lon = 133.7751312
            text = list(sa2_spatial['SA2_NAME'])
            zoom = 2.8
        elif selected_state != 'Australia' and selected_sa4 == 'Any':
            lat = list(sa2_spatial[sa2_spatial.STATE_NAME == selected_state]['lat'])  
            lon = list(sa2_spatial[sa2_spatial.STATE_NAME == selected_state]['lon'])
            center_lat =  np.mean(lat)
            center_lon = np.mean(lon) 
            text = list(sa2_spatial[sa2_spatial.STATE_NAME == selected_state]['SA2_NAME'])
            zoom = 5
        else:
            lat = list(sa2_spatial[sa2_spatial.SA4_NAME_2016 == selected_sa4]['lat'])  
            lon = list(sa2_spatial[sa2_spatial.SA4_NAME_2016 == selected_sa4]['lon'])
            center_lat =  np.mean(lat)
            center_lon = np.mean(lon)  
            text = list(sa2_spatial[sa2_spatial.SA4_NAME_2016 == selected_sa4]['SA4_NAME_2016'])
            zoom = 7

        return {
                    'data': [{
                        'lat': lat,
                        'lon': lon,
                        'type': 'scattermapbox',
                        'text': text,
                        'selected': {
                            'marker': {'color': '#85144b'}
                        }
                    }],
                    'layout': {
                        'mapbox': {
                            'center': {
                                'lat': center_lat,
                                'lon': center_lon                    
                            },
                            'zoom': zoom,
                            'accesstoken': 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2oyY2M4YW55MDF1YjMzbzhmemIzb290NiJ9.sT6pncHLXLgytVEj21q43A'
                        },
                        'margin': {'l': 0, 'r': 0, 't': 0, 'b': 0}
                    }
                }
    else:
        lat = list(sa2_spatial[sa2_spatial.STATE_NAME == 'Victoria']['lat'])  
        lon = list(sa2_spatial[sa2_spatial.STATE_NAME == 'Victoria']['lon'])       
        return {
                    'data': [{
                        'lat': lat,
                        'lon': lon,
                        'type': 'scattermapbox',
                        'text': text,
                        'selected': {
                            'marker': {'color': '#85144b'}
                        }
                    }],
                    'layout': {
                        'mapbox': {
                            'center': {
                                'lat': -25.2743988,
                                'lon': 133.7751312                    
                            },
                            'zoom': 2.8,
                            'accesstoken': 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2oyY2M4YW55MDF1YjMzbzhmemIzb290NiJ9.sT6pncHLXLgytVEj21q43A'
                        },
                        'margin': {'l': 0, 'r': 0, 't': 0, 'b': 0}
                    }
                }       
    
    

#---------------------------------------------------------------------------------
# VISUALISE PLOTS
#---------------------------------------------------------------------------------



@app.callback(
    [Output('sa2-dropdown', 'options'),
     Output('sa2-dropdown', 'value')
    ],
    [Input('data_show', 'data')])
def set_sa2_options(data):
    data_score = pd.DataFrame.from_dict(data)   
    sa2_selected = []
    if not data_score.empty:        
        sa2_selected = list(data_score['Areas Name'])
        
    if len(sa2_selected) > 0:
        return [[{'label': i, 'value': i} for i in sa2_selected],sa2_selected[0]]
    else:
        return [[{'label':'','value':''}],'']


@app.callback(
    Output('deviation', 'figure'),
    [Input('sa2-dropdown', 'value'),
     Input('intermediate-value', 'children')     
    ])
def plot_deviation(value,json_data):
    try:
        datasets = json.loads(json_data)
        df_results = pd.read_json(datasets['df_results'], orient='split')
        feature_border = datasets['feature_border']          
        
        df_temp = df_results.copy()
        
        expectation = []
        for feature in feature_rename:  
            feature_min = df_temp[feature].min()
            feature_max = df_temp[feature].max()
            df_temp[feature] = df_temp.apply(lambda x: (x[feature] - feature_min) / (feature_max - feature_min),axis=1) 
            expectation_value = (feature_border[feature+'_value'] - feature_min) / (feature_max - feature_min)
            expectation.append(expectation_value)

        actual = list(df_temp[df_temp['Areas Name'] == value].iloc[0,1:].values)
            

        return {
                    'data': [
                        {
                            'x': feature_rename,
                            'y': actual,
                            'text': feature_rename,
                            'name': 'Actual',
                            'mode': 'markers',
                            'marker': {'size': 12}
                        },
                        {
                            'x': feature_rename,
                            'y':  expectation,
                            'text': feature_rename,
                            'name': 'Expectation',
                            'mode': 'markers',
                            'marker': {'size': 12}
                        }
                    ],
                    'layout': {
                        'clickmode': 'event+select'
                    }
                }
    except: 
        return {
                    'data': [
                        {
                            'x': [''],
                            'y': [0],
                            'text': [''],
                            'name': 'Actual',
                            'mode': 'markers',
                            'marker': {'size': 12}
                        },
                        {
                            'x': [''],
                            'y':  [0],
                            'text': [''],
                            'name': 'Expectation',
                            'mode': 'markers',
                            'marker': {'size': 12}
                        }
                    ],
                    'layout': {
                        'clickmode': 'event+select'
                    }
                }


 
    

#---------------------------------------------------------------------------------
# COMPARISON TAB
#---------------------------------------------------------------------------------


@app.callback(
    Output('feature1-dropdown', 'options'),
    [Input('feature-dropdown0', 'value')])
def set_cities_options1(selected_cat1):
    return [{'label': i, 'value': i} for i in features_dict[selected_cat1]]

@app.callback(
    Output('feature1-dropdown', 'value'),
    [Input('feature1-dropdown', 'options')])
def set_cities_value1(available_options):
    return available_options[0]['value']

@app.callback(
    Output('display-selected-features1', 'children'),
    [Input('feature-dropdown0', 'value'),
     Input('feature1-dropdown', 'value')])
def set_display_children1(selected_cat1, selected_feature1):
    return u'Feature 1: {} - {}'.format(
        selected_cat1, selected_feature1
    )

#---------------------------------------------------------------------------------
@app.callback(
    Output('feature2-dropdown', 'options'),
    [Input('feature-dropdown1', 'value')])
def set_cities_options1(selected_cat2):
    return [{'label': i, 'value': i} for i in features_dict[selected_cat2]]

@app.callback(
    Output('feature2-dropdown', 'value'),
    [Input('feature2-dropdown', 'options')])
def set_cities_value1(available_options):
    return available_options[0]['value']

@app.callback(
    Output('display-selected-features2', 'children'),
    [Input('feature-dropdown1', 'value'),
     Input('feature2-dropdown', 'value')])
def set_display_children1(selected_cat2, selected_feature2):
    return u'Feature 2: {} - {}'.format(
        selected_cat2, selected_feature2
    )

#---------------------------------------------------------------------------------

@app.callback(
    Output('feature3-dropdown', 'options'),
    [Input('feature-dropdown2', 'value')])
def set_cities_options1(selected_cat3):
    return [{'label': i, 'value': i} for i in features_dict[selected_cat3]]

@app.callback(
    Output('feature3-dropdown', 'value'),
    [Input('feature3-dropdown', 'options')])
def set_cities_value1(available_options):
    return available_options[0]['value']

@app.callback(
    Output('display-selected-features3', 'children'),
    [Input('feature-dropdown2', 'value'),
     Input('feature3-dropdown', 'value')])
def set_display_children1(selected_cat3, selected_feature3):
    return u'Feature 3: {} - {}'.format(
        selected_cat3, selected_feature3
    )
    

#---------------------------------------------------------------------------------
                                          

operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]


def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]

                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part

                # word operators need spaces after them in the filter string,
                # but we don't want these later
                return name, operator_type[0].strip(), value

    return [None] * 3


@app.callback(
    Output('table-paging-with-graph', "data"),
    [Input('table-paging-with-graph', "page_current"),
     Input('table-paging-with-graph', "page_size"),
     Input('table-paging-with-graph', "sort_by"),
     Input('table-paging-with-graph', "filter_query"),
     Input('feature-dropdown0', 'value'),
     Input('feature1-dropdown', 'value'),     
     Input('feature-dropdown1', 'value'),
     Input('feature2-dropdown', 'value'),     
     Input('feature-dropdown2', 'value'),
     Input('feature3-dropdown', 'value')
    ])
def update_table(page_current, page_size, sort_by, filter,input1,input2,input3,input4,input5,input6):
    filtering_expressions = filter.split(' && ')

    input_2 = '_'.join(input2.split())
    input_4 = '_'.join(input4.split())
    input_6 = '_'.join(input6.split())      
    feature_1 = '_'.join([input1,input_2])
    feature_2 = '_'.join([input3,input_4])
    feature_3 = '_'.join([input5,input_6])
    df = get_df(data,feature_1,feature_2,feature_3)    
    
    dff = df
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)

        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            # these operators match pandas series operator method names
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
            # this is a simplification of the front-end filtering logic,
            # only works with complete fields in standard format
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]

    if len(sort_by):
        dff = dff.sort_values(
            [col['column_id'] for col in sort_by],
            ascending=[
                col['direction'] == 'asc'
                for col in sort_by
            ],
            inplace=False
        )

    return dff.iloc[ 
        page_current*page_size: (page_current + 1)*page_size
    ].to_dict('records')


@app.callback(
    Output('table-paging-with-graph-container', "children"),
    [Input('table-paging-with-graph', "data")])
def update_graph(rows):
    dff = pd.DataFrame(rows)

    featurecols = list(dff.keys())
    featurecols.remove('Areas Name')
    featurecols.remove('Areas State')

    return html.Div(
        [
            dcc.Graph(
                id=column,
                figure={
                    "data": [
                        {
                            "x": dff['Areas Name'],
                            "y": dff[column] if column in dff else [],
                            "type": "bar",
                            "marker": {"color": "#0074D9"},
                        }
                    ],
                    "layout": {
                        "xaxis": {"automargin": True},
                        "yaxis": {"automargin": True},
                        "height": 250,
                        "margin": {"t": 10, "l": 10, "r": 10},
                    },
                },
            )
            for column in featurecols
        ]
    )


#---------------------------------------------------------------------------------
# TAB 3 City Profile
#---------------------------------------------------------------------------------

#---------------------------------------------------------------------------------
# AREA SELECTION
#---------------------------------------------------------------------------------


@app.callback(
    Output('tab3_sa4-dropdown', 'options'),
    [Input('tab3_state-dropdown', 'value')])
def set_cities_options2(selected_state):
    return [{'label': i, 'value': i} for i in sa4_name2[selected_state]]


@app.callback(
    Output('tab3_sa4-dropdown', 'value'),
    [Input('tab3_sa4-dropdown', 'options')])
def set_cities_value2(available_options):
    return available_options[0]['value']


@app.callback(
    Output('tab3_sa2-dropdown', 'options'),
    [Input('tab3_sa4-dropdown', 'value')])
def set_cities_options2(selected_sa4):
    return [{'label': i, 'value': i} for i in sa2_name[selected_sa4]]


@app.callback(
    Output('tab3_sa2-dropdown', 'value'),
    [Input('tab3_sa2-dropdown', 'options')])
def set_cities_value2(available_options):
    return available_options[0]['value']


@app.callback(
    [dash.dependencies.Output('data_cat_all', 'data'),
     dash.dependencies.Output('data_cat_all', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected0(value):
    if value is not None:  
        cat_trans = cat_combine[cat_combine.SA2_NAME == value].transpose().reset_index().iloc[1:,:]
        cat_per_sa2 = pd.DataFrame()
        cat_per_sa2['Feature'] = feature_rename
        cat_per_sa2['Australia'] = list(cat_trans.iloc[:8,1])
        cat_per_sa2['State'] = list(cat_trans.iloc[8:16,1])
        cat_per_sa2['SA4'] = list(cat_trans.iloc[16:,1])        
        

        columns = [{'id': c, 'name': c} for c in cat_per_sa2.columns]            
    else:
        cat_per_sa2 = pd.DataFrame()
        columns = []
        
    return [cat_per_sa2.to_dict('records'),columns]


@app.callback(
    [dash.dependencies.Output('data_cat_complete', 'data'),
     dash.dependencies.Output('data_cat_complete', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected00(value):
    if value is not None:  
        cat_trans = cat_combine_all[cat_combine_all.SA2_NAME == value].transpose().reset_index().iloc[1:,:]
        cat_per_sa2 = pd.DataFrame()        
        cat_per_sa2['Feature'] = complete_features       
        cat_per_sa2['Feature'] = cat_per_sa2.apply(lambda x: ' '.join(x.Feature.split('_')) , axis=1)
        cat_per_sa2['Australia'] = list(cat_trans.iloc[:145,1])
        cat_per_sa2['State'] = list(cat_trans.iloc[145:290,1])
        cat_per_sa2['SA4'] = list(cat_trans.iloc[290:,1])     
        

        columns = [{'id': c, 'name': c} for c in cat_per_sa2.columns]            
    else:
        cat_per_sa2 = pd.DataFrame()
        columns = []
        
    return [cat_per_sa2.to_dict('records'),columns]

        
@app.callback(
    [dash.dependencies.Output('data_selected_show1', 'data'),
     dash.dependencies.Output('data_selected_show1', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected1(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_housevalue].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]


@app.callback(
    [dash.dependencies.Output('data_selected_show2', 'data'),
     dash.dependencies.Output('data_selected_show2', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected2(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_unitvalue].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]



@app.callback(
    [dash.dependencies.Output('data_selected_show3', 'data'),
     dash.dependencies.Output('data_selected_show3', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected3(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_demography].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]




@app.callback(
    [dash.dependencies.Output('data_selected_show4', 'data'),
     dash.dependencies.Output('data_selected_show4', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected4(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_population].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]



@app.callback(
    [dash.dependencies.Output('data_selected_show5', 'data'),
     dash.dependencies.Output('data_selected_show5', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_income].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]


@app.callback(
    [dash.dependencies.Output('data_selected_show6', 'data'),
     dash.dependencies.Output('data_selected_show6', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_education].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]


@app.callback(
    [dash.dependencies.Output('data_selected_show7', 'data'),
     dash.dependencies.Output('data_selected_show7', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_employment].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]


@app.callback(
    [dash.dependencies.Output('data_selected_show8', 'data'),
     dash.dependencies.Output('data_selected_show8', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_family].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]


@app.callback(
    [dash.dependencies.Output('data_selected_show9', 'data'),
     dash.dependencies.Output('data_selected_show9', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_health].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]

@app.callback(
    [dash.dependencies.Output('data_selected_show10', 'data'),
     dash.dependencies.Output('data_selected_show10', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_language].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]



@app.callback(
    [dash.dependencies.Output('data_selected_show11', 'data'),
     dash.dependencies.Output('data_selected_show11', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_development].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]

@app.callback(
    [dash.dependencies.Output('data_selected_show12', 'data'),
     dash.dependencies.Output('data_selected_show12', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_business].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]

@app.callback(
    [dash.dependencies.Output('data_selected_show13', 'data'),
     dash.dependencies.Output('data_selected_show13', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_volunteer].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]

@app.callback(
    [dash.dependencies.Output('data_selected_show14', 'data'),
     dash.dependencies.Output('data_selected_show14', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_finance].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]

@app.callback(
    [dash.dependencies.Output('data_selected_show15', 'data'),
     dash.dependencies.Output('data_selected_show15', 'columns')
    ],
    [     
     dash.dependencies.Input('tab3_sa2-dropdown', 'value') 
    ])
def update_table_selected5(value):
    if value is not None:    
        data_select = data_twitter_all[data_twitter_all.SA2_NAME == value ][features_indigenous].transpose().reset_index()
        c_name = list(data_select.columns)
        
        data_select = data_select.rename(columns={c_name[0]:'Features',c_name[1]:'Value'})
        
        data_select['Value'] = data_select.apply(lambda x: rescale(x) ,axis=1)
        data_select['Features'] = data_select.apply(lambda x: ' '.join(x.Features.split("_")) ,axis=1)
        
        columns = [{'id': c, 'name': c} for c in data_select.columns]            
    else:
        data_select = pd.DataFrame()
        columns = []
        
    return [data_select.to_dict('records'),columns]


def rescale(row):
    rescale = row.Value 
    if rescale <= 1:
        rescale = rescale*100
        rescale = round(rescale,2)
    return round(rescale,2)

@app.callback(
    dash.dependencies.Output('map_live', 'figure'),
    [dash.dependencies.Input('tab3_sa2-dropdown', 'value')])
def display_map2(value):

    if value is not None:    
        lat = list(sa2_spatial[sa2_spatial.SA2_NAME == value]['lat'])  
        lon = list(sa2_spatial[sa2_spatial.SA2_NAME == value]['lon'])


        return {
                    'data': [{
                        'lat': lat,
                        'lon': lon,
                        'type': 'scattermapbox',
                        'mode': 'markers',
                        'text':value,
                        'marker': go.scattermapbox.Marker({
                                    'size':30,
                                    'color':'red',
                                    'symbol':'marker'                            

                                   }),
                    }],
                    'layout': {
                        'mapbox': {
                            'center': {
                                'lat': lat[0],
                                'lon': lon[0]                    
                            },
                            'zoom': 10,
                            'accesstoken': 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2oyY2M4YW55MDF1YjMzbzhmemIzb290NiJ9.sT6pncHLXLgytVEj21q43A'
                        },
                        'margin': {'l': 0, 'r': 0, 't': 0, 'b': 0}
                    }
                }
    else:
        return {
                    'data': [{
                        'lat': [],
                        'lon': [],
                        'type': 'scattermapbox',
                    }],
                    'layout': {
                        'mapbox': {
                            'center': {
                                'lat': -25.2743988,
                                'lon': 133.7751312                    
                            },
                            'zoom': 2.8,
                            'accesstoken': 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2oyY2M4YW55MDF1YjMzbzhmemIzb290NiJ9.sT6pncHLXLgytVEj21q43A'
                        },
                        'margin': {'l': 0, 'r': 0, 't': 0, 'b': 0}
                    }
                }
    
@app.callback(
    Output('display-sentiment', 'children'),
    [Input('tab3_sa2-dropdown', 'value')])

def set_display_children11(value):
    score = str(list(data_twitter_all[data_twitter_all.SA2_NAME == value]['Score'])[0])
    if score is not None:
        sentiment = str(list(data_twitter_all[data_twitter_all.SA2_NAME == value]['sentiment'])[0])
        mark_text1 = f'''###### Sentiment Polarity is {sentiment} with average score {score}'''
        return dcc.Markdown(children=mark_text1)    
    else:
        mark_text = f'''###### Not enough Twitter data to calculate Sentiment Polarity in this area'''
        return dcc.Markdown(children=mark_text)  
          
    
@app.callback(
    Output('topic', 'figure'),
    [Input('tab3_sa2-dropdown', 'value')])
def plot_topic(value):
    try:
        topic = data_twitter_all[data_twitter_all.SA2_NAME == value]['topic']

        if topic is not None:
            topic_dict = dict(list(data_twitter_all[data_twitter_all.SA2_NAME == value]['topic'])[0])
            x = list(topic_dict.keys())
            y = list(topic_dict.values())

            return    { 'data' :[{
                                    'x':x, 
                                    'y':y,
                                    'text':y,
                                    'textposition':'auto',
                                }]
                       }
    except:
            return    { 'data' :[{
                                    'x':[], 
                                    'y':[],
                                    'text':[],
                                    'textposition':'auto',
                                }]
                       }


app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/dZVMbK.css"
})
    

if __name__ == '__main__':
    app.run_server(debug=True, port=8053, dev_tools_hot_reload=False, use_reloader=False)
                    



Running on http://127.0.0.1:8053/
Running on http://127.0.0.1:8053/
Running on http://127.0.0.1:8053/
Running on http://127.0.0.1:8053/
Debugger PIN: 759-312-342
Debugger PIN: 759-312-342
Debugger PIN: 759-312-342
Debugger PIN: 759-312-342
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


/Users/indah/anaconda3/lib/python3.7/site-packages/dash/resources.py:49: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/dZVMbK.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.ly/external-resources



Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 487, cost: 4012.0
Run 1, iteration: 2/100, moves: 39, cost: 3981.0
Run 1, iteration: 3/100, moves: 0, cost: 3981.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 194, cost: 4072.0
Run 2, iteration: 2/100, moves: 102, cost: 3992.0
Run 2, iteration: 3/100, moves: 4, cost: 3992.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1065, cost: 4084.0
Run 3, iteration: 2/100, moves: 10, cost: 4084.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 420, cost: 4197.0
Run 4, iteration: 2/100, moves: 0, cost: 4197.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 208, cost: 4393.0
Run 5, iteration: 2/100, moves: 1, cost: 4393.0
Best run 